# Thinking like Xarray : High-level computational patterns

**Deepak Cherian, CGD**

March 9, 2022

-----

## Motivation / Learning goals

From https://toolz.readthedocs.io/en/latest/control.html
> The Toolz library contains dozens of patterns like map and groupby. Learning a core set (maybe a dozen) covers the vast majority of common programming tasks often done by hand.
> A rich vocabulary of core control functions conveys the following benefits:
>    - You identify new patterns
>    - You make fewer errors in rote coding
>    - You can depend on well tested and benchmarked implementations


The same is true for xarray

## Xarray's high-level patterns


Xarray allows you to leverage dataset metadata to write more readable analysis code. The metadata is stored with the data; not in your head.
1. Dimension names: `dim="latitude"` instead of `axis=0`
2. Coordinate "labels": or axis tick labels. `data.sel(latitude=45)` instead of `data[10]`


Xarray also provides high-level computational patterns that cover many data analysis tasks.

1. `rolling` :
   [Operate on rolling windows of your data e.g. running mean](https://xarray.pydata.org/en/stable/computation.html#rolling-window-operations)
1. `coarsen` :
   [Downsample your data](https://xarray.pydata.org/en/stable/computation.html#coarsen-large-arrays)
1. `groupby` :
   [Bin data in to groups and reduce](https://xarray.pydata.org/en/stable/groupby.html)
1. `groupby_bins`: GroupBy after discretizing a numeric variable.
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://xarray.pydata.org/en/stable/time-series.html#resampling-and-grouped-operations)
1. `weighted` :
   [Weight your data before reducing](https://xarray.pydata.org/en/stable/computation.html#weighted-array-reductions)


## Load example dataset

In [ ]:
import numpy as np
import xarray as xr

xr.set_options(keep_attrs=True)

da = xr.tutorial.load_dataset("air_temperature", engine="netcdf4").air
monthly = da.resample(time="M").mean()
data = da.isel(time=0)
data.plot()

## Concept: "index space" vs "label space"


These are windowed operations with a window of a fixed size.

- ``rolling``: sliding window operations e.g. running mean
- ``coarsen``: decimating; reshaping

In [ ]:
data

In [ ]:
# index space
data[10, :]  # 10th element along the first axis; ¯\_(ツ)_/¯

# index space
data.isel(lat=10)  # slightly better, 10th element in latitude

# "label" space
data.sel(lat=50)  # much better! lat=50°N

In [ ]:
# What I wanted to do
data.sel(lat=50)

# What I had to do (if I wasn't using xarray)
data[10, :]

## Xarray provides high-level patterns in both "index space" and  "label space"

#### Index space
1. `rolling` :
   [Operate on rolling windows of your data e.g. running mean](https://xarray.pydata.org/en/stable/computation.html#rolling-window-operations)
1. `coarsen` :
   [Downsample your data](https://xarray.pydata.org/en/stable/computation.html#coarsen-large-arrays)
   
#### Label space
1. `groupby` :
   [Bin data in to groups and reduce](https://xarray.pydata.org/en/stable/groupby.html)
1. `groupby_bins`: GroupBy after discretizing a numeric variable.
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://xarray.pydata.org/en/stable/time-series.html#resampling-and-grouped-operations)

## Index space: windows of fixed width


### Sliding windows of fixed length: ``rolling``

- returns object of same shape as input
- pads with NaNs to make this happen
- supports multiple dimensions

In [ ]:
data.rolling(lat=5, lon=5, center=True).mean().plot()

#### Apply an existing numpy-only function with ``reduce``

Here's an example function: `np.ptp`

In [ ]:
np.ptp?

**Exercise** Calculate the rolling peak-to-peak amplitude in 5 point bins along both latitude and longitude

Answer:

In [ ]:
data.rolling(lat=5, lon=5, center=True).reduce(np.ptp).plot()

#### For more complicated analysis, construct a new array with a new dimension.
Allows things like short-time fourier transform, spectrogram, windowed rolling etc.

In [ ]:
simple = xr.DataArray(np.arange(10), dims="time", coords={"time": np.arange(10)})

# adds a new dimension "window"
simple.rolling(time=5, center=True).construct("window")

``construct`` is clever. It constructs a **view** of the original array, so it is memory-efficient. but you didn't have to know that

**Exercise** Calculate the 5 point running mean in time using `rolling.construct`

Answer

In [ ]:
(
    simple
    .rolling(time=5, center=True)
    .construct("window")
    .mean("window")
)

#### Another ``construct`` example

This is a 2D rolling example; we need to provide two new dimension names

In [ ]:
(
    data
    .rolling(lat=5, lon=5, center=True)
    .construct(lat="lat_roll", lon="lon_roll")
)

### Block windows of fixed length: ``coarsen``

For non-overlapping windows or "blocks" use ``coarsen``. The syntax is very similar to `rolling`. You will need to specify ``boundary`` if the length of the dimension is not a multiple of the block size

In [ ]:
(
    data
    .coarsen(lat=5, lon=5, boundary="trim")
    .mean()
    .plot()
)

#### coarsen supports ``reduce`` for custom reductions

**Exercise** Use ``coarsen.reduce`` to apply `np.ptp` in 5x5 (latxlon) point blocks of `data`

Answer

In [ ]:
(
    data.coarsen(lat=5, lon=5, boundary="trim")
    .reduce(np.ptp)
    .plot()
)

#### coarsen supports ``construct`` for block reshaping

This is usually a good alternative to `np.reshape`

A simple example

In [ ]:
months = xr.DataArray(np.tile(np.arange(1, 13), reps=2), dims="time", coords={"time": np.arange(1,25)})
months

In [ ]:
months.coarsen(time=12).construct(time=("year", "month"))

Note two things:
1. The `time` dimension was also reshaped.
1. The new dimensions `year` and `month` don't have any coordinate labels associated with them.

**Exercise** Reshape the `time` dimension of `monthly` to year x month and visualize the seasonal cycle for two years at 250°E

Answer

In [ ]:
# splits time dimension into year x month
year_month = monthly.coarsen(time=12).construct(time=("year", "month"))
year_month["year"] = np.unique(monthly.time.dt.year)
# seasonal cycle for two years
year_month.sel(lon=250).plot.contourf(col="year", x="month", y="lat", robust=True)

## Label space "windows" or bins : GroupBy

Generalization of ``coarsen``: sometimes the windows you want are not regular.

- ``groupby``: e.g. climatologies, composites; works when "groups" are exact: e.g. characters or integers; not floats
- ``groupby_bins``: binning operations e.g. histograms
- ``resample``: groupby but specialized for time grouping (so far)

**tip** Both `groupby_bins` and `resample` are implemented as `GroupBy` with a specific way of constructing group labels.


### Deconstructing GroupBy


Commonly called "split-apply-combine". 

1. "split" : break dataset into groups
1. "apply" : apply an operation, usually a reduction like `mean`
1. "combine" : concatenate results from apply step along new "group" dimension

But really there is a first step: "identifying groups" also called "factorization". Usually this is the hard part




### Constructing group labels

These functions are useful for constructing "group labels" in many cases

1. [``numpy.digitize``](https://numpy.org/doc/stable/reference/generated/numpy.digitize.html) (binning)
1. [``numpy.searchsorted``](https://numpy.org/doc/stable/reference/generated/numpy.searchsorted.html) supports many other data types
1. [``pandas.factorize``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html) supports characters, strings etc.
1. [``pandas.cut``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) for binning


Xarray uses `pandas.factorize` for `groupby` and `pandas.cut` for `groupby_bins`

#### More commonly useful are ["datetime components"](https://xarray.pydata.org/en/stable/user-guide/time-series.html#datetime-components)

Accessed using ``DataArray.dt.*``

In [ ]:
da.time.dt.day

In [ ]:
da.time.dt.month

In [ ]:
da["time.month"]

#### `floor`, `ceil` and `round` time

In [ ]:
# remove roundoff error in timestamps
# floor to daily frequency
da.time.dt.floor("D")

#### `strftime` can be extremely useful

So useful and so unintuitive that it has its own website: https://strftime.org/

In [ ]:
da.time.dt.strftime("%b-%d")

### Simple Examples

#### groupby

In [ ]:
(
    da.groupby("time.month")
    .mean()
    .plot(col="month", col_wrap=4)
)

#### groupby_bins

Like groupby but now the groups are bins. Useful for histogramming / grouping by `float` variables

In [ ]:
binned = data.groupby_bins("lat", bins=[20, 35, 40, 45, 50])
binned

In [ ]:
binned.mean().plot.line(hue="lat_bins")

#### resample

groupby but specialized for time-grouping to a specified frequency

Simple frequency strings are `"D"` for Day, `"M"` for MonthEnd , `"A"` for Annual etc.

More complicated frequency strings are [available](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases), thanks to Pandas.

All these work with CFTime and Datetime (so all calendars are supported)

In [ ]:
da.resample(time="M").mean()

### Looping over groupby objects is possible

Maybe you want to plot data in each group separately?

In [ ]:
for label, group in da.groupby("time.month"):
    print(label)

This is a DataArray contain data for all December days

In [ ]:
group

### In most cases, avoid a for loop using ``map``

Apply functions that expect xarray Datasets or DataArrays.

Avoid having to manually combine results using concat

In [ ]:
def iqr(da):
    """ Calculates interquartile range """
    return (da.quantile(q=0.75) - da.quantile(q=0.25)).rename("iqr")


da.groupby("time.month").map(iqr)

### groupby supports `reduce` for custom reductions

This applies to `groupby_bins` and `resample`

In [ ]:
(
    da.groupby("time.month")
    .reduce(np.ptp)
    .plot(col="month", col_wrap=4)
)

**tip** `map` is for functions that expect and return xarray objects (see also ``Dataset.map``). `reduce` is for functions that expect and return plain arrays (like numpy or scipy functions)

---

## Summary

Xarray provides methods for high-level analysis patterns:
1. `rolling` :
   [Operate on rolling windows of your data e.g. running mean](https://xarray.pydata.org/en/stable/computation.html#rolling-window-operations)
1. `coarsen` :
   [Downsample your data](https://xarray.pydata.org/en/stable/computation.html#coarsen-large-arrays)
1. `groupby` :
   [Bin data in to groups and reduce](https://xarray.pydata.org/en/stable/groupby.html)
1. `groupby_bins`: GroupBy after discretizing a numeric variable.
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://xarray.pydata.org/en/stable/time-series.html#resampling-and-grouped-operations)
1. `weighted` :
   [Weight your data before reducing](https://xarray.pydata.org/en/stable/computation.html#weighted-array-reductions)


## More resources

1. More tutorials here:https://xarray-contrib.github.io/xarray-tutorial/
1. Answers to common questions on "how to do X" are here: https://xarray.pydata.org/en/stable/howdoi.html